# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [135]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [136]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from dotenv import load_dotenv

In [137]:
# TODO: Load environment variables
load_dotenv()

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

True

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [138]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

from lib.tooling import tool
import os
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="chromadb")

embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("CHROMA_OPENAI_API_KEY"),
    model_name="text-embedding-3-small"
)

collection = chroma_client.get_collection(
    name="udaplay",
    embedding_function=embedding_fn
)

@tool
def retrieve_game(query: str) -> list:
    """
    Semantic search: Finds most relevant results in the vector DB.

    Args:
        query (str): a natural language question about games

    Returns:
        list: A list of dictionaries, each containing:
            - Name
            - Platform
            - YearOfRelease
            - Description
    """
    results = collection.query(query_texts=[query], n_results=3)

    docs = results.get("metadatas", [[]])[0] 
    return [
        {
            "Name": doc.get("Name"),
            "Platform": doc.get("Platform"),
            "YearOfRelease": doc.get("YearOfRelease"),
            "Description": doc.get("Description")
        }
        for doc in docs
    ]

In [139]:
# Test 
sample_query = "What Pokémon games were released on Game Boy?"
results = retrieve_game(sample_query)

from pprint import pprint
pprint(results)

[{'Description': 'Second-generation Pokémon games introducing new regions, '
                 'Pokémon, and gameplay mechanics.',
  'Name': 'Pokémon Gold and Silver',
  'Platform': 'Game Boy Color',
  'YearOfRelease': 1999},
 {'Description': 'Third-generation Pokémon games set in the Hoenn region, '
                 'featuring new Pokémon and double battles.',
  'Name': 'Pokémon Ruby and Sapphire',
  'Platform': 'Game Boy Advance',
  'YearOfRelease': 2002},
 {'Description': 'A classic platformer where Mario embarks on a quest to save '
                 'Princess Toadstool and Dinosaur Land from Bowser.',
  'Name': 'Super Mario World',
  'Platform': 'Super Nintendo Entertainment System (SNES)',
  'YearOfRelease': 1990}]


#### Evaluate Retrieval Tool

In [140]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
import importlib
import lib.evaluation
importlib.reload(lib.evaluation)

from lib.tooling import tool
from lib.evaluation import EvaluationReport
from lib.llm import LLM

@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question.
    
    Args:
    - question: original question from user
    - retrieved_docs: list of game documents from vector DB
    
    Returns:
    - EvaluationReport object containing:
        - useful: bool
        - description: str
    """
    llm = LLM()

    system_prompt = (
        "Your task is to evaluate if the retrieved documents are sufficient to answer the user query.\n"
        "Be critical. Only mark them as useful if they directly help answer the question.\n"
        "Explain why they are or aren't sufficient."
    )

    user_prompt = {
        "question": question,
        "retrieved_docs": retrieved_docs
    }
    
    prompt = f"""
    System: {system_prompt}

    User Question: {question}

    Retrieved Documents:
    {retrieved_docs}
    """

    result = llm.invoke(
    input=prompt,
    response_format=EvaluationReport
    )               

    return result

In [141]:
report = evaluate_retrieval(
    question="When was God of War Ragnarok released?",
    retrieved_docs=[
        {
            "Name": "God of War Ragnarok",
            "Platform": "PlayStation 5",
            "YearOfRelease": 2022,
            "Description": "The sequel to 2018's God of War, following Kratos and Atreus in Norse mythology."
        }
    ]
)

#print report
from pydantic import BaseModel
import json

parsed = json.loads(report.content)  
print(json.dumps(parsed, indent=2))

{
  "useful": true,
  "description": "The retrieved document provides the release year of 'God of War Ragnarok' as 2022, which directly answers the user's question about when the game was released. Additionally, it includes relevant context about the game being a sequel to the 2018 'God of War' and its focus on Norse mythology, which may enhance the user's understanding of the title."
}


In [142]:
!pip install tavily-python

In [143]:
print("TavilyClient successfully imported")

TavilyClient successfully imported


#### Game Web Search Tool

In [144]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
from lib.tooling import tool
from tavily import TavilyClient
import os

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

@tool
def game_web_search(question: str) -> list:
    """
    Web search fallback: Uses Tavily client to search the web.
    
    Args:
    - question: a question about game industry
    
    Returns:
    - A list of search result summaries (dicts) with title, URL, and snippet
    """
    search_response = tavily_client.search(
        query=question,
        search_depth="advanced",
        include_answer=True,
        max_results=5
    )
    
    results = search_response["results"]
    for i, r in enumerate(results, 1):
        print(f"Result {i}:\nTitle: {r['title']}\nURL: {r['url']}\nSnippet: {r['content'][:300]}...\n")
    
    return results

In [145]:
# Test 
results = game_web_search("When was Mortal Kombat X released for PlayStation 5?")

for idx, r in enumerate(results, 1):
    print(f"\nResult {idx}:")
    print(f"Title: {r['title']}")
    print(f"URL: {r['url']}")
    print(f"Snippet: {r['content'][:300]}...\n")

Result 1:
Title: Mortal Kombat X (PS5) 4K 60FPS HDR Gameplay - YouTube
URL: https://www.youtube.com/watch?v=HUNMZwDdrvY
Snippet: Mortal Kombat X[b] is a 2015 fighting game developed by NetherRealm Studios and published by Warner Bros. Interactive Entertainment for Windows, PlayStation 4, and Xbox One. It is the tenth main installment in the Mortal Kombat series and a sequel to Mortal Kombat (2011), taking place 25 years later...

Result 2:
Title: Mortal Kombat X - Wikipedia
URL: https://en.wikipedia.org/wiki/Mortal_Kombat_X
Snippet: An upgraded version of _Mortal Kombat X_, titled _Mortal Kombat XL_,[\[c\]]( was released on March 1, 2016, for PlayStation 4 and Xbox One, including all downloadable content characters from the two released Kombat Packs, almost all bonus alternate costumes available at the time of release, improved...

Result 3:
Title: Mortal Kombat X on PS5? : r/MortalKombat - Reddit
URL: https://www.reddit.com/r/MortalKombat/comments/15can50/mortal_kombat_x_on_ps5/
Snipp

### Agent

In [146]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
from lib.tooling import tool

tools = [retrieve_game, evaluate_retrieval, game_web_search]
agent = Agent(
    model_name="gpt-4", 
    instructions="You are an expert video game research assistant. Answer with relevant game facts.",
    tools=tools
)

run = agent.invoke("When was Pokémon Gold and Silver released?")

run = agent.invoke("When was Pokémon Gold and Silver released?")

final = run.get_final_state()
print(final["messages"][-1].content)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Pokémon Gold and Silver was released in 1999 for the Game Boy Color.


In [147]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
questions = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?",
]

for question in questions:
    print(f"\nQuestion: {question}")
    run = agent.invoke(question)
    final = run.get_final_state()
    print(f"Answer: {final['messages'][-1].content}")


Question: When was Pokémon Gold and Silver released?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer: Pokémon Gold and Silver were released in 1999 for the Game Boy Color.

Question: Which one was the first 3D platformer Mario game?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Answer: The first 3D platformer Mario game was Super Mario 64, released on the Nintendo 64 in 1996.

Question: Was Mortal Kombat X released for Playstation 5?
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMa

### (Optional) Advanced

In [148]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes